In [92]:
## Standard libraries
import os
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.auto import tqdm

/var/folders/f2/gq8xgm9x491cqwtct_tgpy0h0000gn/T/ipykernel_3991/2052077244.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [93]:
import jax
import jax.numpy as jnp
print("Using jax", jax.__version__)

Using jax 0.4.26


In [94]:
a = jnp.zeros((2,5), dtype=jnp.float32)
print(a)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [95]:
b = jnp.arange(6)
print(b)

[0 1 2 3 4 5]


In [96]:
b.__class__

jaxlib.xla_extension.ArrayImpl

In [97]:
b.device

<bound method ArrayImpl.device of Array([0, 1, 2, 3, 4, 5], dtype=int32)>

In [98]:
b_cpu = jax.device_get(b)
print(b_cpu.__class__)

<class 'numpy.ndarray'>


In [99]:
b_gpu = jax.device_put(b_cpu)
print(f'Device put: {b_gpu.__class__} on {b_gpu.devices()}')

Device put: <class 'jaxlib.xla_extension.ArrayImpl'> on {METAL(id=0)}


In [100]:
b_cpu + b_gpu

Array([ 0,  2,  4,  6,  8, 10], dtype=int32)

In [101]:
b_new = b.at[0].set(1)
print('Original array:', b)
print('Changed array:', b_new)

Original array: [0 1 2 3 4 5]
Changed array: [1 1 2 3 4 5]


In [102]:
rng = jax.random.PRNGKey(42)

In [103]:
jax_random_number_1 = jax.random.normal(rng)
jax_random_number_2 = jax.random.normal(rng)
print('JAX - Random number 1:', jax_random_number_1)
print('JAX - Random number 2:', jax_random_number_2)

JAX - Random number 1: -0.9350606
JAX - Random number 2: -0.9350606


In [104]:
np.random.seed(42)
np_random_number_1 = np.random.normal()
np_random_number_2 = np.random.normal()
print('NumPy - Random number 1:', np_random_number_1)
print('NumPy - Random number 2:', np_random_number_2)

NumPy - Random number 1: 0.4967141530112327
NumPy - Random number 2: -0.13826430117118466


In [105]:
rng, subkey1, subkey2 = jax.random.split(rng,num=3) 

In [106]:
#no splitting. I get the same numbers always
jax_random_number_0 = jax.random.normal(rng)
jax_random_number_1 = jax.random.normal(subkey1)
jax_random_number_2 = jax.random.normal(subkey2)
print('JAX new - Random number 0:', jax_random_number_0)
print('JAX new - Random number 1:', jax_random_number_1)
print('JAX new - Random number 2:', jax_random_number_2)

JAX new - Random number 0: -0.5484217
JAX new - Random number 1: -0.18267898
JAX new - Random number 2: 0.7080024


In [107]:
#every time i run this cell i well get different values
rng, subkey1, subkey2 = jax.random.split(rng,num=3) 
jax_random_number_0 = jax.random.normal(rng)
jax_random_number_1 = jax.random.normal(subkey1)
jax_random_number_2 = jax.random.normal(subkey2)
print('JAX new - Random number 0:', jax_random_number_0)
print('JAX new - Random number 1:', jax_random_number_1)
print('JAX new - Random number 2:', jax_random_number_2)

JAX new - Random number 0: -0.26986617
JAX new - Random number 1: -0.4367844
JAX new - Random number 2: -0.082964614


## Tutorial 101

In [108]:
x = jnp.arange(10)
x

Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [109]:
long_vector = jnp.arange(int(1e8))
long_vector.__class__

jaxlib.xla_extension.ArrayImpl

In [110]:
%timeit jnp.dot(long_vector, long_vector).block_until_ready()

4.46 ms ± 488 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
cpu_long_vector = jax.device_get(long_vector)
cpu_long_vector.__class__

numpy.ndarray

In [112]:
%timeit jnp.dot(long_vector, long_vector).block_until_ready()

3.27 ms ± 616 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [113]:
def sum_of_squares(x):
    return jnp.sum(x**2)

In [114]:
sum_of_squares_dx = jax.grad(sum_of_squares)
x = jnp.asarray([1.0,2.0,3.0,4.0])
print(x.__class__)
print(sum_of_squares(x))
print(sum_of_squares_dx(x))

<class 'jaxlib.xla_extension.ArrayImpl'>
30.0
[2. 4. 6. 8.]


In [115]:
def sum_squared_error(x,y):
    return jnp.sum((x-y)**2)
sum_squared_error_dx = jax.grad(sum_squared_error)
y = jnp.asarray([1.1,2.1,3.1,4.1])
print(sum_squared_error(x,y))
print(sum_squared_error_dx(x,y))

0.039999947
[-0.20000005 -0.19999981 -0.19999981 -0.19999981]


In [116]:
jax.grad(sum_squared_error, argnums=(0, 1))(x, y)  # Find gradient wrt both x & y

(Array([-0.20000005, -0.19999981, -0.19999981, -0.19999981], dtype=float32),
 Array([0.20000005, 0.19999981, 0.19999981, 0.19999981], dtype=float32))

In [117]:
def simple_graph(x):
    x = x + 2
    x = x ** 2
    x = x + 3
    y = x.mean()
    return y

In [118]:
inp = jnp.arange(3, dtype=jnp.float32)
inp.__class__

jaxlib.xla_extension.ArrayImpl

In [119]:
print('Input', inp)
print('Output', simple_graph(inp))

Input [0. 1. 2.]
Output 12.666667


In [120]:
jax.make_jaxpr(simple_graph)(inp)

{ lambda ; a:f32[3]. let
    b:f32[3] = add a 2.0
    c:f32[3] = integer_pow[y=2] b
    d:f32[3] = add c 3.0
    e:f32[] = reduce_sum[axes=(0,)] d
    f:f32[] = div e 3.0
  in (f,) }

In [121]:
global_list = []

# Invalid function with side-effect
def norm(x):
    global_list.append(x) #does not contain any operation for this one
    x = x ** 2
    n = x.sum()
    n = jnp.sqrt(n)
    return n

jax.make_jaxpr(norm)(inp)

{ lambda ; a:f32[3]. let
    b:f32[3] = integer_pow[y=2] a
    c:f32[] = reduce_sum[axes=(0,)] b
    d:f32[] = sqrt c
  in (d,) }

In [122]:
grad_function = jax.grad(simple_graph)
gradients = grad_function(inp)
print('Gradient', gradients)

Gradient [1.3333334 2.        2.6666667]


In [123]:
jax.make_jaxpr(grad_function)(inp)

{ lambda ; a:f32[3]. let
    b:f32[3] = add a 2.0
    c:f32[3] = integer_pow[y=2] b
    d:f32[3] = integer_pow[y=1] b
    e:f32[3] = mul 2.0 d
    f:f32[3] = add c 3.0
    g:f32[] = reduce_sum[axes=(0,)] f
    _:f32[] = div g 3.0
    h:f32[] = div 1.0 3.0
    i:f32[3] = broadcast_in_dim[broadcast_dimensions=() shape=(3,)] h
    j:f32[3] = mul i e
  in (j,) }

In [124]:
# when we just want to get the value
val_grad_function = jax.value_and_grad(simple_graph)
val_grad_function(inp)

(Array(12.666667, dtype=float32),
 Array([1.3333334, 2.       , 2.6666667], dtype=float32))

In [125]:
jitted_function = jax.jit(simple_graph)

In [126]:
rng, normal_rng = jax.random.split(rng)


In [127]:
large_input = jax.random.normal(normal_rng, (1000,))
_ = jitted_function(large_input)

In [128]:
%%timeit
simple_graph(large_input).block_until_ready()

787 µs ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [129]:
%%timeit
jitted_function(large_input).block_until_ready()

279 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [130]:
jitted_grad_function = jax.jit(grad_function)
_ = jitted_grad_function(large_input) 

In [131]:
%%timeit
grad_function(large_input).block_until_ready()

3.35 ms ± 65.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
%%timeit
jitted_grad_function(large_input).block_until_ready()

238 µs ± 5.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [133]:
from flax import linen as nn


In [134]:
class SimpleClassifier(nn.Module):
    num_hidden : int   # Number of hidden neurons
    num_outputs : int  # Number of output neurons

    def setup(self):
        # Create the modules we need to build the network
        # nn.Dense is a linear layer
        self.linear1 = nn.Dense(features=self.num_hidden)
        self.linear2 = nn.Dense(features=self.num_outputs)

    def __call__(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = nn.tanh(x)
        x = self.linear2(x)
        return x

In [136]:
#simpler way of creating a simple classifier
class SimpleClassifierCompact(nn.Module):
    num_hidden : int  
    num_outputs : int

    @nn.compact  # Tells Flax to look for defined submodules
    def __call__(self, x):
        # Perform the calculation of the model to determine the prediction
        # while defining necessary layers
        x = nn.Dense(features=self.num_hidden)(x)
        x = nn.tanh(x)
        x = nn.Dense(features=self.num_outputs)(x)
        return x

In [137]:
#creating instance of the model
model = SimpleClassifier(num_hidden=8,num_outputs=1)
print(model)

SimpleClassifier(
    # attributes
    num_hidden = 8
    num_outputs = 1
)


In [141]:
rng, inp_rng, init_rng = jax.random.split(rng,3)

In [145]:

inp = jax.random.normal(inp_rng,(8,2))
inp

Array([[-1.3439181 ,  0.6429476 ],
       [-1.5676413 , -0.44147977],
       [ 0.49920234, -0.36310518],
       [ 0.8277361 , -0.02077403],
       [ 0.4741161 , -0.35662487],
       [-2.3667781 ,  0.9393318 ],
       [ 0.16964886,  0.37653938],
       [-0.5358841 ,  0.19641948]], dtype=float32)

In [149]:
params = model.init(init_rng,inp)
print(params)

{'params': {'linear1': {'kernel': Array([[ 1.024376  , -1.052183  , -0.1483135 ,  1.2987319 ,  0.3865956 ,
        -1.0295712 , -0.2703718 , -0.12079524],
       [-0.52960914,  0.7966237 ,  0.89103186, -1.1658981 ,  0.95768267,
         0.7459659 ,  0.5387747 , -1.2564553 ]], dtype=float32), 'bias': Array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)}, 'linear2': {'kernel': Array([[ 0.43645597],
       [-0.39781412],
       [ 0.06989034],
       [-0.08034126],
       [ 0.6816833 ],
       [ 0.48783168],
       [ 0.67914635],
       [ 0.18095477]], dtype=float32), 'bias': Array([0.], dtype=float32)}}}


In [155]:
#apply the model on an input
#one output neuron in the model for all 8 batch elements
model.apply(params,inp)

Array([[ 0.17609759],
       [-0.53406435],
       [-0.1307149 ],
       [ 0.19904903],
       [-0.13222754],
       [ 0.20863602],
       [ 0.32784808],
       [-0.00445377]], dtype=float32)